In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, time

!pip install pycaret
from pycaret.regression import * # 다양한 머신러닝 모델을 비교하기 위한 pycaret

from sklearn.preprocessing import MinMaxScaler # 데이터 스케일링
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor # 머신러닝 모델 중 Extra Trees Regressor 사용

In [2]:
#pandas 기본 세팅
pd.set_option('display.float_format', None)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_rows', 300)

In [3]:
# 데이터를 읽을 경로
path = './data/'

In [4]:
# 파일 명 저장
file_list = os.listdir(path[:-1])
print(file_list)

['stocks.csv', 'trade_train.csv']


In [5]:
# 파일 읽기
original_stocks = pd.read_csv(path+file_list[0],index_col=['index'])
original_trade = pd.read_csv(path+file_list[1],index_col=0)

In [6]:
trade = original_trade.copy()
stocks = original_stocks.copy()

In [7]:
# 차후 결과를 검증해야할 코드 선정
original_test_list = stocks.loc[stocks['20년7월TOP3대상여부']=='Y','종목번호'].unique().tolist()

In [8]:
len(original_test_list)

135

In [9]:
y_list = original_test_list.copy()

# 데이터 전처리 작업

## Stocks data 중 str형을 int형으로 범주화

In [10]:
stocks.head()

,기준일자,종목번호,종목명,20년7월TOP3대상여부,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,종목시가,종목고가,종목저가,종목종가,거래량,거래금액_만원단위
index,,,,,,,,,,,,,,
0,20190701,A000020,동화약품,N,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,9680,9840,9680,9710,31341,30581
1,20190701,A000080,하이트진로,N,코스피,제조업,음료 제조업,알코올음료 제조업,20550,20550,20200,20300,127186,258535
2,20190701,A000100,유한양행,Y,코스피,제조업,의료용 물질 및 의약품 제조업,의약품 제조업,259000,267500,250000,251500,395757,10139357
3,20190701,A000120,CJ대한통운,N,코스피,운수 및 창고업,육상운송 및 파이프라인 운송업,도로 화물 운송업,136500,137000,133000,136000,62513,846100
4,20190701,A000140,하이트진로홀딩스,Y,코스피,금융 및 보험업,금융업,기타 금융업,9060,9190,8910,9000,7661,6896


### 시장구분

In [11]:
stocks['시장구분'].value_counts()

코스닥    188307
코스피     97754
Name: 시장구분, dtype: int64

In [12]:
stocks['시장구분'] = stocks['시장구분'].apply(lambda x: 1 if x=='코스닥' else 0)

### 대분류

In [13]:
stocks['표준산업구분코드_대분류'].value_counts()

제조업                        185875
정보통신업                       37102
도매 및 소매업                    21172
금융 및 보험업                    14107
전문· 과학 및 기술 서비스업            10496
건설업                          5628
운수 및 창고업                     5517
수도· 하수 및 폐기물 처리· 원료 재생업      1608
서비스업                         1340
교육 서비스업                      1072
사업시설관리 및 사업지원 및 임대 서비스업       804
예술· 스포츠 및 여가관련                536
전기· 가스· 증기 및 공기조절 공급업         536
개인 서비스업                       268
Name: 표준산업구분코드_대분류, dtype: int64

In [14]:
big_cate = stocks['표준산업구분코드_대분류'].unique().tolist()

In [15]:
stocks['표준산업구분코드_대분류'] = stocks['표준산업구분코드_대분류'].apply(lambda x: big_cate.index(x))

In [16]:
stocks['표준산업구분코드_대분류'].value_counts()

0     185875
6      37102
5      21172
2      14107
3      10496
4       5628
1       5517
9       1608
11      1340
12      1072
13       804
10       536
7        536
8        268
Name: 표준산업구분코드_대분류, dtype: int64

### 중분류

각 카테고리 별로 중분류를 각각 범주화 진행

In [17]:
mid_cate = [
    stocks.loc[stocks['표준산업구분코드_대분류']==big_num,'표준산업구분코드_중분류'].unique().tolist()
    for big_num in range(len(big_cate))
]

In [18]:
for big_num in range(len(big_cate)):
    stocks.loc[stocks['표준산업구분코드_대분류']==big_num,'표준산업구분코드_중분류'] = stocks.loc[stocks['표준산업구분코드_대분류']==big_num,'표준산업구분코드_중분류'].apply(lambda x: mid_cate[big_num].index(x))

### 소분류

중분류와 마찬가지로 각각 범주화 진행

In [19]:
for big_num in range(len(big_cate)):
    for mid_num in range(len(mid_cate[big_num])):
        small_cate = stocks.loc[(stocks['표준산업구분코드_대분류']==big_num)&(stocks['표준산업구분코드_중분류']==mid_num),'표준산업구분코드_소분류'].unique().tolist()
        stocks.loc[(stocks['표준산업구분코드_대분류']==big_num)&(stocks['표준산업구분코드_중분류']==mid_num),'표준산업구분코드_소분류'] = stocks.loc[(stocks['표준산업구분코드_대분류']==big_num)&(stocks['표준산업구분코드_중분류']==mid_num),'표준산업구분코드_소분류'].apply(lambda x: small_cate.index(x))

### Stocks의 Column 정리

In [20]:
stocks = stocks.drop(columns=['20년7월TOP3대상여부']).copy()

In [21]:
stocks_col_name = stocks.columns.tolist()

In [22]:
stocks_col_name

['기준일자',
 '종목번호',
 '종목명',
 '시장구분',
 '표준산업구분코드_대분류',
 '표준산업구분코드_중분류',
 '표준산업구분코드_소분류',
 '종목시가',
 '종목고가',
 '종목저가',
 '종목종가',
 '거래량',
 '거래금액_만원단위']

In [23]:
stocks_col_name[4] = '대분류'
stocks_col_name[5] = '중분류'
stocks_col_name[6] = '소분류'
stocks_col_name[7] = '시가'
stocks_col_name[8] = '고가'
stocks_col_name[9] = '저가'
stocks_col_name[10] = '종가'
stocks_col_name[12] = '거래금액'

In [24]:
stocks.columns = stocks_col_name

In [25]:
stocks.head()

,기준일자,종목번호,종목명,시장구분,대분류,중분류,소분류,시가,고가,저가,종가,거래량,거래금액
index,,,,,,,,,,,,,
0,20190701,A000020,동화약품,0,0,0,0,9680,9840,9680,9710,31341,30581
1,20190701,A000080,하이트진로,0,0,1,0,20550,20550,20200,20300,127186,258535
2,20190701,A000100,유한양행,0,0,0,0,259000,267500,250000,251500,395757,10139357
3,20190701,A000120,CJ대한통운,0,1,0,0,136500,137000,133000,136000,62513,846100
4,20190701,A000140,하이트진로홀딩스,0,2,0,0,9060,9190,8910,9000,7661,6896


In [26]:
# 중간 데이터 손실을 방지한 정보 저장
stocks_temp = stocks.copy()
trade_temp = trade.copy()

In [27]:
stocks = stocks_temp.copy()
trade = trade_temp.copy()

In [28]:
stocks['기준년월'] = stocks['기준일자'].apply(lambda x:int(x/100))

# Trade를 기반으로한 mart base 생성

In [29]:
key_col = ['기준년월','종목번호']

In [30]:
mart = trade[key_col].drop_duplicates().copy()

In [31]:
concat_mart = pd.DataFrame([[202007]*len(y_list),y_list]).T
concat_mart.columns = mart.columns

In [32]:
mart = pd.concat([mart,concat_mart]).copy()
mart = pd.merge(mart,stocks.iloc[:,1:7],on='종목번호',how='left').drop_duplicates().copy()

In [33]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류
0,201907,A000660,SK하이닉스,0.0,0.0,3,0
268,201907,A001820,삼화콘덴서,0.0,0.0,10,0
536,201907,A004020,현대제철,0.0,0.0,6,0
804,201907,A005380,현대차,0.0,0.0,2,0
1072,201907,A005930,삼성전자,0.0,0.0,10,1


## 결측치 처리 작업
* 상장 폐지로 인해 유실된 데이터
* 종목시가=0인 데이터
* 액면 변화로 인해 변동이 큰 데이터
* 각 기준년월 별 종목번호 당 데이터 길이가 7일이 안되는 row 제거

### 시장구분 & 대분류가 int에서 float으로 변환된 원인 파악
* 종목번호 A204990 상장 폐지로 인한 데이터 유실

In [34]:
mart.loc[(mart['시장구분']>=0)==False]

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류
170952,201908,A204990,NaN,NaN,NaN,NaN,NaN
203006,201909,A204990,NaN,NaN,NaN,NaN,NaN
379734,201910,A204990,NaN,NaN,NaN,NaN,NaN
588084,201912,A204990,NaN,NaN,NaN,NaN,NaN


* 종목번호 A204990 제거

In [35]:
mart = mart.loc[mart['종목번호']!='A204990'].copy()

In [36]:
for i in mart.columns[3:5].tolist():
    mart[i] = mart[i].astype(int).copy()
mart.index = range(len(mart))

In [37]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류
0,201907,A000660,SK하이닉스,0,0,3,0
1,201907,A001820,삼화콘덴서,0,0,10,0
2,201907,A004020,현대제철,0,0,6,0
3,201907,A005380,현대차,0,0,2,0
4,201907,A005930,삼성전자,0,0,10,1


### 시가=0 인 data 제거
* 거래정지/ 액면 변화/ 상장 폐지 데이터 제거

In [38]:
stocks = stocks.loc[stocks['시가']!=0].copy()

### 액면분할 및 액면병합으로 인해서 데이터의 변동이 큰 데이터 제거

In [39]:
print('세미콘라이트')
display(stocks[(stocks['종목번호']=='A214310')&(stocks['기준년월']==202001)].iloc[-2:,:])

print('오리엔트바이오')
display(stocks[(stocks['종목번호']=='A002630')&(stocks['기준년월']==202002)].iloc[1:3,:])

print('유한양행')
display(stocks[(stocks['종목번호']=='A000100')&(stocks['기준년월']==202004)].iloc[1:3,:])

세미콘라이트


,기준일자,종목번호,종목명,시장구분,대분류,중분류,소분류,시가,고가,저가,종가,거래량,거래금액,기준년월
index,,,,,,,,,,,,,,
137500,20200109,A214310,세미콘라이트,1,0,3,0,473,603,464,542,30205107,1699408,202001
152584,20200131,A214310,세미콘라이트,1,0,3,0,2550,2775,2500,2575,782292,205607,202001


오리엔트바이오


,기준일자,종목번호,종목명,시장구분,대분류,중분류,소분류,시가,고가,저가,종가,거래량,거래금액,기준년월
index,,,,,,,,,,,,,,
153896,20200204,A002630,오리엔트바이오,0,0,0,0,309,313,300,306,7414383,226780,202002
167911,20200221,A002630,오리엔트바이오,0,0,0,0,750,797,665,676,9751042,692161,202002


유한양행


,기준일자,종목번호,종목명,시장구분,대분류,중분류,소분류,시가,고가,저가,종가,거래량,거래금액,기준년월
index,,,,,,,,,,,,,,
199248,20200402,A000100,유한양행,0,0,0,0,224500,229000,217000,224500,71427,1591008,202004
203584,20200408,A000100,유한양행,0,0,0,0,51000,52900,46550,46550,2612972,12855508,202004


In [40]:
mart = mart[(mart['종목번호']!='A214310')|(mart['기준년월']!=202001)].copy()
mart = mart[(mart['종목번호']!='A002630')|(mart['기준년월']!=202002)].copy()
mart = mart[(mart['종목번호']!='A000100')|(mart['기준년월']!=202004)].copy()
mart = mart.sort_values(key_col)
mart.index = range(len(mart))

### 각 기준년월 별 종목번호 당 데이터 길이가 7일이 안되는 row 제거
* 학습에 미치는 영향이 미미한 종목 혹은 상장폐지 된 종목이 학습에서 제외되도록 설정

In [41]:
drop_index = pd.merge(mart[key_col],stocks[key_col+['시가']],on=key_col,how='left').groupby(key_col).count().reset_index().copy()
drop_index = drop_index.sort_values(key_col)
drop_index = drop_index[drop_index['시가']<7].index.tolist()

In [42]:
mart = mart.drop(index=drop_index).copy()
mart.index = range(len(mart))

In [43]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류
0,201907,A000080,하이트진로,0,0,1,0
1,201907,A000100,유한양행,0,0,0,0
2,201907,A000120,CJ대한통운,0,1,0,0
3,201907,A000140,하이트진로홀딩스,0,2,0,0
4,201907,A000145,하이트진로홀딩스우,0,2,0,0


## 각 기준년월 별 종목번호 당 테마 추가
직접 조사해가며 테마 딕셔너리 작성

* **대선** : 이낙연, 이재명 등 영향력 있는 정치인 관련주
* **코로나** : 코로나 관련 제품 종목 (EX. 마스크, 손 소독제, 진단키트 등)
* **언택트** : 포스트 코로나 시대로 인한 언택트 사업 증가에 따른 수혜주
* **백신** : 코로나 백신 관련주
* **미래산업** : 반도체, 디지털 뉴딜, 수소차, 2차전지 등 미래산업 관련주, 삼성 퀄컴 최신 스냅드래곤 875G 삼성전자 파운드리로 양산 예정에 따른 수혜주
* **인플루언서** : 정세균 국무총리 지하 고속도로 개발 언급, 새만금 국제공항 건설 추진, 이외에도 BTS 등 사회적 이슈로 혜택을 받은 종목들
* **돼지열병_흑사병** : 기본적인 예상되는 팬데믹에 따른 수혜주 외에도 돼지육에 대한 불신에 따른 대체육 분야 등 포함
* **애국** : 일본과 계속되는 경제갈등에 반응하여 일본불매 운동 등의 활동이력 등이 있는 종목
* **한한령** : 중국서 한류를 제하는 법(한한령)이 해제된다는 가설에 따른 수혜주
* **대북** : 남북경협 얘기에 따른 농업, 철도, 혹은 탈원전 주 등
* **오일** : 국제 유가 변화에 따른 수혜주

In [44]:
thema = ['대선', '코로나', '언택트', '백신', '미래산업', '인플루언서', '미중무역', '돼지열병_흑사병', '철강', '애국', '한한령', '대북', '오일', '무속성']

def dic_add(stock_name, ind) :
    if stock_name not in stock_dic.keys() : stock_dic[stock_name] = []
    if thema[ind] not in stock_dic[stock_name] : stock_dic[stock_name].append(thema[ind])
        
name_list = stocks['종목명'].unique().tolist()
        
thema_list = [0] * len(thema)
thema_list[0] = ['에이텍', '남선알미늄', '티케이케미칼', '이월드', '한창제지', '티비씨', '덕신하우징', '한국팩키지', '한네트', '한국프랜지', '고려산업', 'KNN', '디지틀조선', '하츠', '벽산', '서연', '서연탑메탈', '모베이스', '모베이스전자', '아이크래프트', '덕성', '덕성우', '태양금속', '남선알미우', '국영지앤엠', '까스텔바작', '서원', '대창', '에쎈테크', 'SDN', '남화토건', '남화산업', '범양건영', '주연테크', '부국철강', '승일', '신신제약', '케이씨티', '코리아나', '이화공영', 'KT서브마린', '중앙에너비스', '장원테크', '캠시스', '정다운', '지엘팜텍', '이루온', '티플랙스', '엠게임', '서린바이오', '우진', '진흥기업', '써니전자', '안랩', '까뮤이앤씨', '오픈베이스', '링네트', '우성사료', '화인베스틸', '보해양조', '풍강', 'YBM넷', '동아지질', '오리엔트바이오', '제이엠아이', '아가방컴퍼니', '대양금속', 'OQP', '부방', '아세아텍', '알에프세미', '이구산업', '국일신동']
thema_list[1] = ['GV', '화승인더', '메지온', 'HRS', '크린앤사이언스', '인피니트헬스케어', '셀바스헬스케어', '서울바이오시스', '엔에스엔', '셀루메드', '제이엠아이', '국동', '오리엔트바이오', '슈프리마', '웰바이오텍', '마크로젠', '나노엔텍', '서린바이오', '국제약품', 'KGETS', '인선이엔티', '와이엔텍', '코엔텍', '대상', '씨티씨바이오', '티앤알바이오팹', '유유제약', '파버나인', '씨유메디칼', '메디아나', '멕아이씨에스', '케이씨티', '동양', '휴메딕스', '셀트리온헬스케어', '세틀뱅크', '케이엠제약', '오공', '깨끗한나라', '모나리자', '케이엠', '톱텍', '웰크론', '엔브이에이치코리아', '창해에탄올', 'MH에탄올', '한국알콜', '인트로메딕', '마이크로디지탈', '우리들휴브레인', '로고스바이오', '필로시스헬스케어', '지노믹트리', '넥스트BT', '인트론바이오', '오상자이엘', '수젠텍', '피씨엘', '녹십자엠에스', '랩지노믹스', '미코', '씨젠', '바디텍메드', '바이오니아', 'EDGC', '레몬', '오텍', 'GH신소재', '우정바이오']
thema_list[2] = ['케어랩스', 'NHN', '팬오션', '에코마케팅', 'NHN한국사이버결제', '서울식품', '피씨디렉트', '네오펙트', '알에프세미', 'KG모빌리언스', '컴퍼니케이', '투비소프트', '인성정보', '한국맥널티', '삼륭물산', '이트론', '이씨에스', '이마트', '디앤씨미디어', '비트컴퓨터', '태림포장', '태평양물산', 'GS리테일', '풀무원', '삼양식품', '삼천리자전거', '콜마비앤에이치', '팜스빌', '뉴트리', '에이치엘사이언스', '노바렉스', '비상교육', '메가스터디', '메가엠디', '아이스크림에듀', 'YBM넷', '아프리카TV', '네스엠', '플레이위드', '네오위즈', '미스터블루', '룽투코리아', '더블유게임즈', '미투온', '드래곤플라이', '선데이토즈', '와이제이엠게임즈', '액션스퀘어', '조이시티', '넷게임즈', '넥슨지티', '펄어비스', '넷마블', '엔씨소프트', '엠게임', '위메이드', '웹젠', '컴투스', '카페24', '코리아센터', '다날', 'KG이니시스', 'NAVER', '카카오']
thema_list[3] = ['파멥신', '삼성바이오로직스', '현대약품', '삼일제약', 'JW신약', '대정화금', '신테카바이오', '셀트리온제약', '펩트론', '테라젠이텍스', '안트로젠', '프로스테믹스', '카이노스메드', '경남제약', '에이프로젠KIC', '에이프로젠H&G', '금호에이치티', '아이큐어', '셀리버리', '에이프로젠제약', '바이온', '인스코비', '에이치엘비', 'SK디스커버리우', 'SV인베스트먼트', '경남제약헬스케어', '비케이탑스', '골드퍼시픽', '메드팩토', '한미사이언스', '유바이오로직스', '삼성제약', '유한양행', '이연제약', '동구바이오제약', '알리코제약', '안국약품', '동성제약', '한올바이오파마', 'CMG제약', '명문제약', '노터스', '뉴지랩', 'JW중외제약', '제일약품', '종근당', '강스템바이오텍', '화일약품', '앱클론', '유틸렉스', '코미팜', '네이처셀', '제노포커스', '영진약품', '대원제약', '경동제약', '신일제약', '한국유니온제약', '일신바이오', 'SK디스커버리', '녹십자웰빙', '녹십자랩셀', '녹십자셀', '진원생명과학', '레고켐바이오', '유나이티드제약', '텔콘RF제약', '진양제약', '크리스탈', '현대바이오', '대웅제약', '대웅', '일양약품', '제넨바이오', '바이넥스', '제넥신', '동화약품', '종근당바이오', '셀트리온', '큐리언트', '부광약품', '대화제약', '비씨월드제약', '신풍제약우', '신풍제약', '라이브플렉스', '휴온스', '아미코젠', '바이오리더스', '바이오톡스텍', '휴마시스', '진매트릭스', '바른손이앤에이', '파미셀', '에이비프로바이오', '바른손', '시노펙스', '에스맥', '녹십자', '엔지켐생명과학', 'SK케미칼우', 'SK케미칼', '아이센스', '젬백스']
thema_list[4] = ['원익큐브', '신성이엔지', '한화솔루션', '시그네틱스', '파인텍', '신흥에스이씨', '대한광통신', '우수AMS', '싸이맥스', '에이디테크놀로지', '모바일어플라이언스', '세경하이테크', '더존비즈온', '한온시스템', '하츠', '코위버', '아이에이네트웍스', '에코프로', '이노메트리', '한샘', '가비아', 'SK하이닉스', '윌링스', '씨아이에스', '평화산업', '데일리블록체인', '이엔에프테크놀로지', '미스터블루', '소프트센', '러셀', '엠게임', '케이엔제이', 'APS홀딩스', '성창오토텍', '계양전기', 'AP시스템', '덱스터', '오이솔루션', '아이앤씨', '씨앤지하이테크', '테스', '넷게임즈', '파워넷', '필옵틱스', '로보로보', '케이엠더블유', '세종텔레콤', '남성', '아이즈비전', '에코바이오', '벽산', '에프알텍', '라온시큐어', '포메탈', '에스넷', '플레이위드', '기산텔레콤', '선데이토즈', 'RFHIC', 'SK머티리얼즈', '현대모비스', '현대글로비스', '유에스티', '이오테크닉스', '엠에스오토텍', '데이타솔루션', '제이스텍', '전파기지국', '두산퓨얼셀', '인텍플러스', '유비쿼스', '쏠리드', '하나머티리얼즈', '룽투코리아', '일진다이아', '천보', '에프엔에스테크', '세원', '삼성SDI', '에프에스티', '마이크로프랜드', '서암기계공업', '파루', '스페코', '우리산업홀딩스', '드림텍', '이노인스트루먼트', '유티아이', '코스모화학', '서진시스템', '영우디에스피', '옴니시스템', '이루온', '삼기오토모티브', '코리아에프티', '에스맥', '기아차', '이엠코리아', '아이쓰리시스템', '삼진엘앤디', '효성첨단소재', '에스코넥', 'LG화학', '디에이테크놀로지', '후성', '모트렉스', '뉴로스', '현대공업', '더블유게임즈', '파인테크닉스', '다우기술', '링네트', '현대위아', '제이엔케이히터', 'KMH하이텍', '솔브레인홀딩스', '테스나', '줌인터넷', '피앤이솔루션', '에스에프에이', '에코프로비엠', 'OCI', '기가레인', '에치에프알', '칩스앤미디어', '대보마그네틱', '삼성전자', '원익QnC', '타이거일렉', '피델릭스', '테크윙', '우리산업', '포스코ICT', '앤씨앤', '오션브릿지', '코오롱머티리얼', '미투온', '펄어비스', '위메이드', '제이앤티씨', '다산네트웍스', '시노펙스', '씨에스베어링', '뉴인텍', '한일단조', '두올', '대우부품', '아이씨디', '지엔코', '케이씨텍', '엔씨소프트', '탑엔지니어링', '아이크래프트', '엘오티베큠', 'KH바텍', '대성엘텍', '월덱스', '와이솔', 'PI첨단소재', '씨에스윈드', '삼지전자', '한미반도체', '에스퓨얼셀', '텔레칩스', '유니크', '유니테스트', '이랜텍', '원익IPS', '상신이디피', '유니퀘스트', '우리로', '조이시티', '나무기술', '주성엔지니어링', '라닉스', 'KG케미칼', '드래곤플라이', 'GST', 'HB테크놀러지', '우리넷', '보성파워텍', '나노캠텍', 'ISC', '와이제이엠게임즈', '유니셈', '피피아이', '현대에너지솔루션', '유비쿼스홀딩스', '유진테크', '상아프론테크', '오픈베이스', '동아화성', 'DB하이텍', '액션스퀘어', '매직마이크로', '원익머트리얼즈', 'SDN', '명성티엔에스', '삼화콘덴서', '넷마블', '뉴파워프라즈마', '삼화전기', '웹젠', '대성파인텍', '코스모신소재', '에이스테크', '피에스케이', '일진머티리얼즈', '머큐리', '상보', '이노와이어리스', 'SKC솔믹스', '엑셈', '링크제니시스', '포스코케미칼', '레이크머티리얼즈', '엘앤에프', '유니슨', '이수페타시스', '현대차', '두산솔루스', '엔에스', '디케이티', '센트랄모텍', '풍국주정', '화신정공', '아모그린텍', '네오위즈', '아남전자', '컴투스', '넥슨지티', '동국S&C', '만도', '하나마이크론', '에스앤에스텍', '영화테크', 'SFA반도체', '위지윅스튜디오', '동운아나텍', '아이티센', '콤텍시스템', '한국전자인증', '한국정보인증', '아톤', '동진쎄미켐', '램테크놀러지', '경인양행', '켐트로스', '3S', '심텍', '마이크로컨텍솔', '두산중공업', '효성중공업', '이엔플러스', 'STX중공업', '혜인', '어보브반도체', '실리콘웍스', '엘비세미콘', '고영', '유니온머티리얼', '나무가', '하이비젼시스템', '주연테크', '지니뮤직', '포비스티앤씨', '삼성전자우', '대덕', 'SK텔레콤', 'KT', '비트컴퓨터', '두산인프라코어', 'SK이노베이션', '한화시스템', '코윈테크', '삼성SDI우', 'LG화학우', '현대차우', '국일제지', '엑사이엔씨', '엘엠에스', '덕양산업', 'UCI', '이더블유케이', '인스코비', '에스모', '두산솔루스1우', '두산솔루스2우B', '두산퓨얼셀1우', '두산퓨얼셀2우B', '메탈라이프', '쎄미시스코', '동화기업', '알에프텍', '인피니트헬스케어', '윈스', '비엠티', '쌍용정보통신', '에스에이엠티', '에스티아이', '서울반도체', '송원산업', '화승인더', '덕산테코피아', '케어랩스', '로보스타']
thema_list[5] = ['삼표시멘트', '아이에이네트웍스', '코오롱', '메디톡스', '이화공영', '삼성엔지니어링', '키이스트', '체시스', '코오롱머티리얼', '케이사인', '한국정보통신', '덕신하우징', '두산중공업', '이지바이오', '케이피엠테크', '뉴로스', '루미마이크로', '범양건영', '코오롱플라스틱', '한국정보인증', '엔바이오니아', '대화제약', '디피씨', '특수건설', '우리기술', 'GS건설', '라온시큐어', '덕양산업', '코센', '백광산업', '지란지교시큐리티', '자연과환경', '한국전자인증', '시큐브', '우원개발', '남광토건', '뉴보텍', 'KG케미칼', '베셀', '대우건설', '상아프론테크', '동아지질', '드림시큐리티', '코웨이', '코오롱생명과학', '초록뱀', 'SG세계물산', '에스텍파마', '고려시멘트', '휴네시온', '한국주철관', '시노펙스', '동양철관', 'KT서브마린', 'SH에너지화학', '텔콘RF제약', '엘비세미콘', '아이크래프트', '코오롱인더', '에스엠', '와이지엔터테인먼트', '큐브엔터', '초록뱀', '키이스트', '디피씨', '스튜디오드래곤', '에이스토리', 'IHQ', '쇼박스', 'NEW', '제이콘텐트리', '덱스터', '위지윅스튜디오', 'JYPEnt', 'CJENM', 'CJCGV', 'SMLifeDesign', 'SMC&C', '마니커에프앤지', '마니커', '체리부로', '하림', '정다운', '동우팜투테이블', '푸드나무', '한성기업', '두올', '마이크로프랜드', 'KC코트렐', '아톤', '한국화장품', '한국화장품제조', '코리아나', '클리오', '토니모리', '본느', '한국콜마', '콜마비앤에이치', '아모레퍼시픽', '아모레G', 'LG생활건강', '코스맥스', '에이블씨엔씨', '애경산업', '세화피앤씨', '제이준코스메틱', 'YGPLUS', '넥스트아이', '브이티지엠피', '신세계인터내셔날', '에스디생명공학', 'SK바이오랜드', '제로투세븐', '클래시스', 'MP한강', '아이티센', '콤텍시스템', '유라테크', '켐트로닉스', '우성사료', '원익큐브', '디케이티', '비에이치', '뉴파워프라즈마', '유티아이', '제이앤티씨', '엘이티', '파인테크닉스', '서진시스템', 'KH바텍', '파인텍', '드림텍', '세경하이테크', '상보', '에스맥', '필옵틱스', '동운아나텍', 'LG이노텍', '덕우전자', '하이비젼시스템', '아이티엠반도체', '삼성전기', '파트론', '나무가', '드림어스컴퍼니', '다나와', '파워로직스', '캠시스', '옵트론텍', '엠씨넥스', '액트로', '해성옵틱스', '레이크머티리얼즈', '경남바이오파마', '인텍플러스', '신신제약', '한독크린텍', 'SK케미칼', 'SK케미칼우', '나노엔텍', 'SK', 'SK우', 'SK디스커버리', 'SK디스커버리우', 'SK네트웍스', 'SK증권', '한국조선해양', '현대미포조선', '대우조선해양', '삼성중공업', '한진중공업', 'HSD엔진', '한국카본', '동성화인텍', '일신바이오', '서린바이오', '삼화전기', '유진기업', '한국비엔씨', '현대차', '기아차', '현대위아', '현대모비스', '코엔텍', '와이엔텍', '인선이엔티', 'KGETS', '케이씨티', '한네트', '케이씨에스', '한국전자금융', '푸른기술', '아이디스', '뉴지랩', '인콘', '켐온', '비덴트', '우리기술투자', 'SCI평가정보', '오스템임플란트', '제노레이', '아시아나IDT', '에어부산', '아시아나항공', '한국항공우주', '한화에어로스페이스', '인텔리안테크', '퍼스텍', '삼성중공우', '에이티넘인베스트', '두산인프라코어', 'SK이노베이션', '크루셜텍', '슈프리마', '디앤씨미디어', '현대차우', '아가방컴퍼니', '에스피시스템스', '세미콘라이트', '오성첨단소재', 'W홀딩컴퍼니', '삼성출판사', '경남제약', '손오공', '안트로젠', '롯데지주', '롯데지주우', '테고사이언스', '켄코아에어로스페이스', 'HRS', '이녹스첨단소재', '팬오션', '아스트']
thema_list[6] = ['유니온', '유니온머티리얼', '샘표식품', 'LG유플러스', '혜인', '티플랙스', '노바텍', '쎄노텍', '수산중공업', '포스코엠텍', '한국내화', '엘컴텍', '쌍방울', '에스맥', '태경비케이', 'KC코트렐', '샘표', '풀무원', '미래생명자원', '현대사료', '한일사료', '우성사료']
thema_list[7] = ['체시스', '이글벳', '우진비앤지', '파루', '체리부로', '제일바이오', '진바이오텍', '중앙백신', '진원생명과학', '대성미생물', '코미팜', '태경비케이', '신라에스지', '우정바이오', '국제약품', '필로시스헬스케어', 'KPX생명과학', '마니커에프앤지', '마니커', '하림', '정다운', '동우팜투테이블', '푸드나무', 'CJ씨푸드', '이지홀딩스', '우리손에프앤지']
thema_list[8] = ['POSCO', '현대제철', '동국제강', '문배철강', '부국철강', '동양철관', '대창솔루션']
thema_list[9] = ['모나미', '후성', '하이트진로홀딩스', '롯데쇼핑', '신성통상', '쌍방울', '코데즈컴바인', '그리티', '보라티알', '토박스코리아', '까스텔바작', '하이트진로홀딩스우', '하이트진로', '리드코프', 'TBH글로벌', '비비안', '삼륭물산', '동진쎄미켐', '경인양행', '램테크놀러지', '메탈라이프', '쎄미시스코', '힘스']
thema_list[10] = ['에스엠', '와이지엔터테인먼트', '큐브엔터', '초록뱀', '키이스트', '디피씨', '스튜디오드래곤', '에이스토리', 'IHQ', '쇼박스', 'NEW', '제이콘텐트리', '덱스터', '위지윅스튜디오', 'JYPEnt', 'CJENM', 'CJCGV', '한국화장품', '한국화장품제조', '코리아나', '클리오', '토니모리', '본느', '한국콜마', '콜마비앤에이치', '아모레퍼시픽', '아모레G', 'LG생활건강', '코스맥스', '에이블씨엔씨', '애경산업', '제이준코스메틱', 'YGPLUS', '넥스트아이', '브이티지엠피', '신세계인터내셔날', '에스디생명공학', 'SK바이오랜드', '제로투세븐', '클래시스', 'MP한강', '팬스타엔터프라이즈', 'SMLifeDesign', 'SMC&C', '아가방컴퍼니', '한스바이오메드']
thema_list[11] = ['대아티아이', '현대로템', '부산산업', '서암기계공업', '대호에이엘', '우원개발', '특수건설', '이화공영', '조비', '경농', '대유', '인디에프', '좋은사람들', '신원', '코데즈컴바인', '한창', '팬스타엔터프라이즈', '일신석재', '빅텍', '남선알미늄', '동양물산', '스페코', '한일단조', '휴니드', '포메탈', '퍼스텍', 'LIG넥스원', '한화에어로스페이스', '한화우', '이디티', '한국테크놀로지', '휴비스', '휴림로봇', '대창솔루션', '에이비프로바이오', '우리기술', '한전KPS', '두산중공업', '우진', '오르비텍', '이트론', '양지사', '에코마이스터', '서희건설', '풍산', '쌍용정보통신', '제룡전기']
thema_list[12] = ['삼성SDI', '대림제지', '삼성SDI우', 'LG화학', 'LG화학우', '롯데케미칼', 'SK이노베이션', 'SK네트웍스', '한화솔루션', '이수화학', '코오롱인더', '효성', '효성첨단소재', '포스코케미칼', 'SKC', '벽산', '한국전력', '아시아나항공', '아시아나IDT', '대한항공', '제주항공', '진에어', 'HDC', 'HDC현대산업개발', '한진칼', '티웨이항공', '티웨이홀딩스', '모두투어', '하나투어', 'CJ대한통운', '현대글로비스', '한진', '한솔로지스틱스', '한익스프레스', '태웅로직스', '동방', 'GS', '지어소프트', '태림포장', '대영포장', '대한항공우', '한진칼우', '흥구석유', 'SOil']

stock_dic = {}
have_prop = set()
for i in thema_list[:-1] : have_prop |= set(i)
thema_list[-1] = list(set(name_list) - have_prop)
for i in range(len(thema_list)) :
    for t in thema_list[i] :
        dic_add(t, i)

In [45]:
# 종목명을 넣으면 해당하는 테마에 관련된 list를 return하는 함수
def stock_them_list(x, thema = thema, stock_dic = stock_dic):
    tm = [0]*len(thema)
    for t in stock_dic[x]:
        tm[thema.index(t)] = 1
    return tm

In [46]:
thema_df = pd.DataFrame(mart['종목명'].apply(lambda x: stock_them_list(x)).tolist()).copy()
thema_df.columns = thema

In [47]:
for i in thema:
    mart[i] = thema_df[i].tolist()

In [48]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,철강,애국,한한령,대북,오일,무속성
0,201907,A000080,하이트진로,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,201907,A000100,유한양행,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,201907,A000120,CJ대한통운,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,201907,A000140,하이트진로홀딩스,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,201907,A000145,하이트진로홀딩스우,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


## 주가, 거래금액 Data갯수가 종목번호마다 다름
### 개당 데이터의 개수를 기준연월 별 25개로 통일

>1. 모자란 갯수는 중간에 평균값을 삽입하여 총 갯수를 25개로 맞춤
>2. 25개를 맞추기 전 min_max 스케일링 및 정규화를 진행
>3. tf Column을 추가하여 해당 날짜가 진짜 값인지 가짜 값인지 구분

In [49]:
min_max = MinMaxScaler()
std = StandardScaler()

standard_col = stocks.columns[7:-1].tolist()
mm_day_col = ["mm"+'_'+i1+'_'+i2
              for i1 in ['종목시가', '종목고가', '종목저가', '종목종가', '거래량', '거래금액_만원단위'] 
              for i2 in ["Day_"+str(i3+1) for i3 in range(25)]]
std_day_col = ["std"+'_'+i1+'_'+i2
               for i1 in ['종목시가', '종목고가', '종목저가', '종목종가', '거래량', '거래금액_만원단위'] 
               for i2 in ["Day_"+str(i3+1) for i3 in range(25)]]+[
    'tf_'+ i1 for i1 in ["Day_"+str(i3+1) for i3 in range(25)]
]

In [50]:
def add_row(data,want_len):
    temp_data = data.copy()
    cut_num = int(len(temp_data)/(want_len-len(temp_data)+1))
    temp_data.index = temp_data.index*10
    
    add_df_list = []
    cnt_i = 1
    
    while len(temp_data)+len(add_df_list)!=want_len:
        add_df = pd.DataFrame(temp_data.iloc[cut_num*cnt_i-1:cut_num*cnt_i+1].mean()).T.copy()
        add_df.index = [int(np.mean(temp_data.iloc[cut_num*cnt_i-1:cut_num*cnt_i+1].index))]
        add_df.iloc[0,-1] = 0
        add_df.iloc[:,-1] = add_df.iloc[:,-1].astype(int)
        add_df_list.append(add_df)
        cnt_i+=1
        
    temp_data = pd.concat([temp_data,pd.concat(add_df_list)]).copy()
    temp_data = temp_data.sort_index().copy()
    temp_data.index = range(len(temp_data))
    return temp_data

def make_row25(data,wnat_len=25):
    temp_data = data.copy()
    if len(temp_data)<13:
        temp_data = add_row(temp_data,13).copy()
    if len(temp_data)<25:
        temp_data = add_row(temp_data,25).copy()
    return temp_data

def make_data_col(i, stocks=stocks, mart=mart, mm_day_col=mm_day_col, std_day_col=std_day_col):
    scale_data = stocks[(stocks['기준년월']==mart.iloc[i,0])&(stocks['종목번호']==mart.iloc[i,1])].iloc[:,6:12].copy()
    min_max_data = pd.DataFrame(min_max.fit_transform(scale_data)).copy()
    std_data = pd.DataFrame(std.fit_transform(scale_data)).copy()
    
    min_max_data['tf']=1
    std_data['tf']=1
    
    add_min_max = make_row25(min_max_data).copy()
    add_std = make_row25(std_data).copy()
    
    add_min_max.columns = [0]*7
    add_std.columns = [0]*7
    
    final = pd.concat([add_min_max.iloc[:,[col]] for col in range(6)]+[add_std.iloc[:,[col]] for col in range(7)]).T.copy()
    final.columns = mm_day_col+std_day_col
    
    return final

In [51]:
mart_n = len(mart)
data_col = [0]*mart_n

for i in range(mart_n):
    if i%100==0:
        st_time = time.time()
        
    data_col[i] = make_data_col(i).copy()
    
    if i%100==0:
        ed_time = time.time()-st_time
        print("\r남은 개수 :",i,"/",mart_n,"\t\t남은 시간 :",round(ed_time*(mart_n-i-1),2),"\t\t",end='')

남은 개수 : 5900 / 5923 		남은 시간 : 1.65 				

In [52]:
data_col = pd.concat(data_col).copy()
data_col.index = range(len(data_col))

In [53]:
for i in mart.columns.tolist():
    data_col[i] = mart[i].tolist()

In [54]:
mart = data_col[data_col.columns.tolist()[-mart.shape[1]:]+data_col.columns.tolist()[:-mart.shape[1]]].copy()

In [55]:
for i in mart.columns[-25:].tolist():
    mart[i] = mart[i].astype(int).copy()

In [56]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,철강,애국,한한령,대북,오일,무속성,mm_종목시가_Day_1,mm_종목시가_Day_2,mm_종목시가_Day_3,mm_종목시가_Day_4,mm_종목시가_Day_5,mm_종목시가_Day_6,mm_종목시가_Day_7,mm_종목시가_Day_8,mm_종목시가_Day_9,mm_종목시가_Day_10,mm_종목시가_Day_11,mm_종목시가_Day_12,mm_종목시가_Day_13,mm_종목시가_Day_14,mm_종목시가_Day_15,mm_종목시가_Day_16,mm_종목시가_Day_17,mm_종목시가_Day_18,mm_종목시가_Day_19,mm_종목시가_Day_20,mm_종목시가_Day_21,mm_종목시가_Day_22,mm_종목시가_Day_23,mm_종목시가_Day_24,mm_종목시가_Day_25,mm_종목고가_Day_1,mm_종목고가_Day_2,mm_종목고가_Day_3,mm_종목고가_Day_4,mm_종목고가_Day_5,mm_종목고가_Day_6,mm_종목고가_Day_7,mm_종목고가_Day_8,mm_종목고가_Day_9,mm_종목고가_Day_10,mm_종목고가_Day_11,mm_종목고가_Day_12,mm_종목고가_Day_13,mm_종목고가_Day_14,mm_종목고가_Day_15,mm_종목고가_Day_16,mm_종목고가_Day_17,mm_종목고가_Day_18,mm_종목고가_Day_19,mm_종목고가_Day_20,mm_종목고가_Day_21,mm_종목고가_Day_22,mm_종목고가_Day_23,mm_종목고가_Day_24,mm_종목고가_Day_25,mm_종목저가_Day_1,mm_종목저가_Day_2,mm_종목저가_Day_3,mm_종목저가_Day_4,mm_종목저가_Day_5,mm_종목저가_Day_6,mm_종목저가_Day_7,mm_종목저가_Day_8,mm_종목저가_Day_9,mm_종목저가_Day_10,mm_종목저가_Day_11,mm_종목저가_Day_12,mm_종목저가_Day_13,mm_종목저가_Day_14,mm_종목저가_Day_15,mm_종목저가_Day_16,mm_종목저가_Day_17,mm_종목저가_Day_18,mm_종목저가_Day_19,mm_종목저가_Day_20,mm_종목저가_Day_21,mm_종목저가_Day_22,mm_종목저가_Day_23,mm_종목저가_Day_24,mm_종목저가_Day_25,mm_종목종가_Day_1,mm_종목종가_Day_2,mm_종목종가_Day_3,mm_종목종가_Day_4,mm_종목종가_Day_5,mm_종목종가_Day_6,mm_종목종가_Day_7,mm_종목종가_Day_8,mm_종목종가_Day_9,mm_종목종가_Day_10,mm_종목종가_Day_11,mm_종목종가_Day_12,mm_종목종가_Day_13,mm_종목종가_Day_14,mm_종목종가_Day_15,mm_종목종가_Day_16,mm_종목종가_Day_17,mm_종목종가_Day_18,mm_종목종가_Day_19,mm_종목종가_Day_20,mm_종목종가_Day_21,mm_종목종가_Day_22,mm_종목종가_Day_23,mm_종목종가_Day_24,mm_종목종가_Day_25,mm_거래량_Day_1,mm_거래량_Day_2,mm_거래량_Day_3,mm_거래량_Day_4,mm_거래량_Day_5,mm_거래량_Day_6,mm_거래량_Day_7,mm_거래량_Day_8,mm_거래량_Day_9,mm_거래량_Day_10,mm_거래량_Day_11,mm_거래량_Day_12,mm_거래량_Day_13,mm_거래량_Day_14,mm_거래량_Day_15,mm_거래량_Day_16,mm_거래량_Day_17,mm_거래량_Day_18,mm_거래량_Day_19,mm_거래량_Day_20,mm_거래량_Day_21,mm_거래량_Day_22,mm_거래량_Day_23,mm_거래량_Day_24,mm_거래량_Day_25,mm_거래금액_만원단위_Day_1,mm_거래금액_만원단위_Day_2,mm_거래금액_만원단위_Day_3,mm_거래금액_만원단위_Day_4,mm_거래금액_만원단위_Day_5,mm_거래금액_만원단위_Day_6,mm_거래금액_만원단위_Day_7,mm_거래금액_만원단위_Day_8,mm_거래금액_만원단위_Day_9,mm_거래금액_만원단위_Day_10,mm_거래금액_만원단위_Day_11,mm_거래금액_만원단위_Day_12,mm_거래금액_만원단위_Day_13,mm_거래금액_만원단위_Day_14,mm_거래금액_만원단위_Day_15,mm_거래금액_만원단위_Day_16,mm_거래금액_만원단위_Day_17,mm_거래금액_만원단위_Day_18,mm_거래금액_만원단위_Day_19,mm_거래금액_만원단위_Day_20,mm_거래금액_만원단위_Day_21,mm_거래금액_만원단위_Day_22,mm_거래금액_만원단위_Day_23,mm_거래금액_만원단위_Day_24,mm_거래금액_만원단위_Day_25,std_종목시가_Day_1,std_종목시가_Day_2,std_종목시가_Day_3,std_종목시가_Day_4,std_종목시가_Day_5,std_종목시가_Day_6,std_종목시가_Day_7,std_종목시가_Day_8,std_종목시가_Day_9,std_종목시가_Day_10,std_종목시가_Day_11,std_종목시가_Day_12,std_종목시가_Day_13,std_종목시가_Day_14,std_종목시가_Day_15,std_종목시가_Day_16,std_종목시가_Day_17,std_종목시가_Day_18,std_종목시가_Day_19,std_종목시가_Day_20,std_종목시가_Day_21,std_종목시가_Day_22,std_종목시가_Day_23,std_종목시가_Day_24,std_종목시가_Day_25,std_종목고가_Day_1,std_종목고가_Day_2,std_종목고가_Day_3,std_종목고가_Day_4,std_종목고가_Day_5,std_종목고가_Day_6,std_종목고가_Day_7,std_종목고가_Day_8,std_종목고가_Day_9,std_종목고가_Day_10,std_종목고가_Day_11,std_종목고가_Day_12,std_종목고가_Day_13,std_종목고가_Day_14,std_종목고가_Day_15,std_종목고가_Day_16,std_종목고가_Day_17,std_종목고가_Day_18,std_종목고가_Day_19,std_종목고가_Day_20,std_종목고가_Day_21,std_종목고가_Day_22,std_종목고가_Day_23,std_종목고가_Day_24,std_종목고가_Day_25,std_종목저가_Day_1,std_종목저가_Day_2,std_종목저가_Day_3,std_종목저가_Day_4,std_종목저가_Day_5,std_종목저가_Day_6,std_종목저가_Day_7,std_종목저가_Day_8,std_종목저가_Day_9,std_종목저가_Day_10,std_종목저가_Day_11,std_종목저가_Day_12,std_종목저가_Day_13,std_종목저가_Day_14,std_종목저가_Day_15,std_종목저가_Day_16,std_종목저가_Day_17,std_종목저가_Day_18,std_종목저가_Day_19,std_종목저가_Day_20,std_종목저가_Day_21,std_종목저가_Day_22,std_종목저가_Day_23,std_종목저가_Day_24,std_종목저가_Day_25,std_종목종가_Day_1,std_종목종가_Day_2,std_종목종가_Day_3,std_종목종가_Day_4,std_종목종가_Day_5,std_종목종가_Day_6,std_종목종가_Day_7,std_종목종가_Day_8,std_종목종가_Day_9,std_종목종가_Day_10,std_종목종가_Day_11,std_종목종가_Day_12,std_종목종가_Day_13,std_종목종가_Day_14,std_종목종가_Day_15,std_종목종가_Day_16,std_종목종가_Day_17,std_종목종가_Day_18,std_종목종가_Day_19,std_종목종가_Day_20,std_종목종가_Day_21,std_종목종가_Day_22,std_종목종가_Day_23,std_종목종가_Day_24,std_종목종가_Day_25,std_거래량_Day_1,std_거래량_Day_2,std_거래량_Day_3,std_거래량

# 마트에 거래량& 거래 대금 값 추가
* 거래량, 거래 대금과 매수고객수와의 연관성 높게 평가

### 마트에 거래량 등급 추가

In [57]:
def trade_vol(i, stocks=stocks, mart=mart):
    dt = stocks[(stocks['기준년월']==mart.iloc[i,0])&(stocks['종목번호']==mart.iloc[i,1])][['거래량']].copy()
    dt['tf']=1
    add_dt = make_row25(dt).copy()
    add_dt['tf'] = (add_dt['거래량']/10000000).astype(int)
    f = add_dt[['tf']].T.copy()
    f.columns = ['거래량_class_Day'+str(i+1) for i in range(25)]
    return f

In [58]:
mart_n = len(mart)
trade_class=[0]*mart_n

for i in range(mart_n):
    if i%100==0:
        st_time = time.time()
        
    trade_class[i] = trade_vol(i)
    
    if i%100==0:
        ed_time = time.time()-st_time
        print("\r남은 개수 :",i,"/",mart_n,"\t\t남은 시간 :",round(ed_time*(mart_n-i-1),2),"\t\t",end='')

남은 개수 : 5900 / 5923 		남은 시간 : 1.56 				

In [59]:
trade_class = pd.concat(trade_class)
trade_class.index = range(len(trade_class))
for i in ['거래량_class_Day'+str(i+1) for i in range(25)]:
    mart[i] = trade_class[i].tolist()

In [60]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,철강,애국,한한령,대북,오일,무속성,mm_종목시가_Day_1,mm_종목시가_Day_2,mm_종목시가_Day_3,mm_종목시가_Day_4,mm_종목시가_Day_5,mm_종목시가_Day_6,mm_종목시가_Day_7,mm_종목시가_Day_8,mm_종목시가_Day_9,mm_종목시가_Day_10,mm_종목시가_Day_11,mm_종목시가_Day_12,mm_종목시가_Day_13,mm_종목시가_Day_14,mm_종목시가_Day_15,mm_종목시가_Day_16,mm_종목시가_Day_17,mm_종목시가_Day_18,mm_종목시가_Day_19,mm_종목시가_Day_20,mm_종목시가_Day_21,mm_종목시가_Day_22,mm_종목시가_Day_23,mm_종목시가_Day_24,mm_종목시가_Day_25,mm_종목고가_Day_1,mm_종목고가_Day_2,mm_종목고가_Day_3,mm_종목고가_Day_4,mm_종목고가_Day_5,mm_종목고가_Day_6,mm_종목고가_Day_7,mm_종목고가_Day_8,mm_종목고가_Day_9,mm_종목고가_Day_10,mm_종목고가_Day_11,mm_종목고가_Day_12,mm_종목고가_Day_13,mm_종목고가_Day_14,mm_종목고가_Day_15,mm_종목고가_Day_16,mm_종목고가_Day_17,mm_종목고가_Day_18,mm_종목고가_Day_19,mm_종목고가_Day_20,mm_종목고가_Day_21,mm_종목고가_Day_22,mm_종목고가_Day_23,mm_종목고가_Day_24,mm_종목고가_Day_25,mm_종목저가_Day_1,mm_종목저가_Day_2,mm_종목저가_Day_3,mm_종목저가_Day_4,mm_종목저가_Day_5,mm_종목저가_Day_6,mm_종목저가_Day_7,mm_종목저가_Day_8,mm_종목저가_Day_9,mm_종목저가_Day_10,mm_종목저가_Day_11,mm_종목저가_Day_12,mm_종목저가_Day_13,mm_종목저가_Day_14,mm_종목저가_Day_15,mm_종목저가_Day_16,mm_종목저가_Day_17,mm_종목저가_Day_18,mm_종목저가_Day_19,mm_종목저가_Day_20,mm_종목저가_Day_21,mm_종목저가_Day_22,mm_종목저가_Day_23,mm_종목저가_Day_24,mm_종목저가_Day_25,mm_종목종가_Day_1,mm_종목종가_Day_2,mm_종목종가_Day_3,mm_종목종가_Day_4,mm_종목종가_Day_5,mm_종목종가_Day_6,mm_종목종가_Day_7,mm_종목종가_Day_8,mm_종목종가_Day_9,mm_종목종가_Day_10,mm_종목종가_Day_11,mm_종목종가_Day_12,mm_종목종가_Day_13,mm_종목종가_Day_14,mm_종목종가_Day_15,mm_종목종가_Day_16,mm_종목종가_Day_17,mm_종목종가_Day_18,mm_종목종가_Day_19,mm_종목종가_Day_20,mm_종목종가_Day_21,mm_종목종가_Day_22,mm_종목종가_Day_23,mm_종목종가_Day_24,mm_종목종가_Day_25,mm_거래량_Day_1,mm_거래량_Day_2,mm_거래량_Day_3,mm_거래량_Day_4,mm_거래량_Day_5,mm_거래량_Day_6,mm_거래량_Day_7,mm_거래량_Day_8,mm_거래량_Day_9,mm_거래량_Day_10,mm_거래량_Day_11,mm_거래량_Day_12,mm_거래량_Day_13,mm_거래량_Day_14,mm_거래량_Day_15,mm_거래량_Day_16,mm_거래량_Day_17,mm_거래량_Day_18,mm_거래량_Day_19,mm_거래량_Day_20,mm_거래량_Day_21,mm_거래량_Day_22,mm_거래량_Day_23,mm_거래량_Day_24,mm_거래량_Day_25,mm_거래금액_만원단위_Day_1,mm_거래금액_만원단위_Day_2,mm_거래금액_만원단위_Day_3,mm_거래금액_만원단위_Day_4,mm_거래금액_만원단위_Day_5,mm_거래금액_만원단위_Day_6,mm_거래금액_만원단위_Day_7,mm_거래금액_만원단위_Day_8,mm_거래금액_만원단위_Day_9,mm_거래금액_만원단위_Day_10,mm_거래금액_만원단위_Day_11,mm_거래금액_만원단위_Day_12,mm_거래금액_만원단위_Day_13,mm_거래금액_만원단위_Day_14,mm_거래금액_만원단위_Day_15,mm_거래금액_만원단위_Day_16,mm_거래금액_만원단위_Day_17,mm_거래금액_만원단위_Day_18,mm_거래금액_만원단위_Day_19,mm_거래금액_만원단위_Day_20,mm_거래금액_만원단위_Day_21,mm_거래금액_만원단위_Day_22,mm_거래금액_만원단위_Day_23,mm_거래금액_만원단위_Day_24,mm_거래금액_만원단위_Day_25,std_종목시가_Day_1,std_종목시가_Day_2,std_종목시가_Day_3,std_종목시가_Day_4,std_종목시가_Day_5,std_종목시가_Day_6,std_종목시가_Day_7,std_종목시가_Day_8,std_종목시가_Day_9,std_종목시가_Day_10,std_종목시가_Day_11,std_종목시가_Day_12,std_종목시가_Day_13,std_종목시가_Day_14,std_종목시가_Day_15,std_종목시가_Day_16,std_종목시가_Day_17,std_종목시가_Day_18,std_종목시가_Day_19,std_종목시가_Day_20,std_종목시가_Day_21,std_종목시가_Day_22,std_종목시가_Day_23,std_종목시가_Day_24,std_종목시가_Day_25,std_종목고가_Day_1,std_종목고가_Day_2,std_종목고가_Day_3,std_종목고가_Day_4,std_종목고가_Day_5,std_종목고가_Day_6,std_종목고가_Day_7,std_종목고가_Day_8,std_종목고가_Day_9,std_종목고가_Day_10,std_종목고가_Day_11,std_종목고가_Day_12,std_종목고가_Day_13,std_종목고가_Day_14,std_종목고가_Day_15,std_종목고가_Day_16,std_종목고가_Day_17,std_종목고가_Day_18,std_종목고가_Day_19,std_종목고가_Day_20,std_종목고가_Day_21,std_종목고가_Day_22,std_종목고가_Day_23,std_종목고가_Day_24,std_종목고가_Day_25,std_종목저가_Day_1,std_종목저가_Day_2,std_종목저가_Day_3,std_종목저가_Day_4,std_종목저가_Day_5,std_종목저가_Day_6,std_종목저가_Day_7,std_종목저가_Day_8,std_종목저가_Day_9,std_종목저가_Day_10,std_종목저가_Day_11,std_종목저가_Day_12,std_종목저가_Day_13,std_종목저가_Day_14,std_종목저가_Day_15,std_종목저가_Day_16,std_종목저가_Day_17,std_종목저가_Day_18,std_종목저가_Day_19,std_종목저가_Day_20,std_종목저가_Day_21,std_종목저가_Day_22,std_종목저가_Day_23,std_종목저가_Day_24,std_종목저가_Day_25,std_종목종가_Day_1,std_종목종가_Day_2,std_종목종가_Day_3,std_종목종가_Day_4,std_종목종가_Day_5,std_종목종가_Day_6,std_종목종가_Day_7,std_종목종가_Day_8,std_종목종가_Day_9,std_종목종가_Day_10,std_종목종가_Day_11,std_종목종가_Day_12,std_종목종가_Day_13,std_종목종가_Day_14,std_종목종가_Day_15,std_종목종가_Day_16,std_종목종가_Day_17,std_종목종가_Day_18,std_종목종가_Day_19,std_종목종가_Day_20,std_종목종가_Day_21,std_종목종가_Day_22,std_종목종가_Day_23,std_종목종가_Day_24,std_종목종가_Day_25,std_거래량_Day_1,std_거래량_Day_2,std_거래량_Day_3,std_거래량

### 마트에 거래량 값 추가

In [61]:
def trade_vol2(i, stocks=stocks, mart=mart):
    dt = stocks[(stocks['기준년월']==mart.iloc[i,0])&(stocks['종목번호']==mart.iloc[i,1])][['거래량']].copy()
    dt['tf']=1
    add_dt = make_row25(dt).copy()
    add_dt['tf'] = (add_dt['거래량']/1000000000)
    f = add_dt[['tf']].T.copy()
    f.columns = ['거래량_float_Day'+str(i+1) for i in range(25)]
    return f

In [62]:
mart_n = len(mart)
trade_float=[0]*mart_n

for i in range(mart_n):
    if i%100==0:
        st_time = time.time()
        
    trade_float[i] = trade_vol2(i)
    
    if i%100==0:
        ed_time = time.time()-st_time
        print("\r남은 개수 :",i,"/",mart_n,"\t\t남은 시간 :",round(ed_time*(mart_n-i-1),2),"\t\t",end='')

남은 개수 : 5900 / 5923 		남은 시간 : 0.99 				

In [63]:
trade_float = pd.concat(trade_float)
trade_float.index = range(len(trade_float))

In [64]:
for i in ['거래량_float_Day'+str(i+1) for i in range(25)]:
    mart[i] = trade_float[i].tolist()

In [65]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,철강,애국,한한령,대북,오일,무속성,mm_종목시가_Day_1,mm_종목시가_Day_2,mm_종목시가_Day_3,mm_종목시가_Day_4,mm_종목시가_Day_5,mm_종목시가_Day_6,mm_종목시가_Day_7,mm_종목시가_Day_8,mm_종목시가_Day_9,mm_종목시가_Day_10,mm_종목시가_Day_11,mm_종목시가_Day_12,mm_종목시가_Day_13,mm_종목시가_Day_14,mm_종목시가_Day_15,mm_종목시가_Day_16,mm_종목시가_Day_17,mm_종목시가_Day_18,mm_종목시가_Day_19,mm_종목시가_Day_20,mm_종목시가_Day_21,mm_종목시가_Day_22,mm_종목시가_Day_23,mm_종목시가_Day_24,mm_종목시가_Day_25,mm_종목고가_Day_1,mm_종목고가_Day_2,mm_종목고가_Day_3,mm_종목고가_Day_4,mm_종목고가_Day_5,mm_종목고가_Day_6,mm_종목고가_Day_7,mm_종목고가_Day_8,mm_종목고가_Day_9,mm_종목고가_Day_10,mm_종목고가_Day_11,mm_종목고가_Day_12,mm_종목고가_Day_13,mm_종목고가_Day_14,mm_종목고가_Day_15,mm_종목고가_Day_16,mm_종목고가_Day_17,mm_종목고가_Day_18,mm_종목고가_Day_19,mm_종목고가_Day_20,mm_종목고가_Day_21,mm_종목고가_Day_22,mm_종목고가_Day_23,mm_종목고가_Day_24,mm_종목고가_Day_25,mm_종목저가_Day_1,mm_종목저가_Day_2,mm_종목저가_Day_3,mm_종목저가_Day_4,mm_종목저가_Day_5,mm_종목저가_Day_6,mm_종목저가_Day_7,mm_종목저가_Day_8,mm_종목저가_Day_9,mm_종목저가_Day_10,mm_종목저가_Day_11,mm_종목저가_Day_12,mm_종목저가_Day_13,mm_종목저가_Day_14,mm_종목저가_Day_15,mm_종목저가_Day_16,mm_종목저가_Day_17,mm_종목저가_Day_18,mm_종목저가_Day_19,mm_종목저가_Day_20,mm_종목저가_Day_21,mm_종목저가_Day_22,mm_종목저가_Day_23,mm_종목저가_Day_24,mm_종목저가_Day_25,mm_종목종가_Day_1,mm_종목종가_Day_2,mm_종목종가_Day_3,mm_종목종가_Day_4,mm_종목종가_Day_5,mm_종목종가_Day_6,mm_종목종가_Day_7,mm_종목종가_Day_8,mm_종목종가_Day_9,mm_종목종가_Day_10,mm_종목종가_Day_11,mm_종목종가_Day_12,mm_종목종가_Day_13,mm_종목종가_Day_14,mm_종목종가_Day_15,mm_종목종가_Day_16,mm_종목종가_Day_17,mm_종목종가_Day_18,mm_종목종가_Day_19,mm_종목종가_Day_20,mm_종목종가_Day_21,mm_종목종가_Day_22,mm_종목종가_Day_23,mm_종목종가_Day_24,mm_종목종가_Day_25,mm_거래량_Day_1,mm_거래량_Day_2,mm_거래량_Day_3,mm_거래량_Day_4,mm_거래량_Day_5,mm_거래량_Day_6,mm_거래량_Day_7,mm_거래량_Day_8,mm_거래량_Day_9,mm_거래량_Day_10,mm_거래량_Day_11,mm_거래량_Day_12,mm_거래량_Day_13,mm_거래량_Day_14,mm_거래량_Day_15,mm_거래량_Day_16,mm_거래량_Day_17,mm_거래량_Day_18,mm_거래량_Day_19,mm_거래량_Day_20,mm_거래량_Day_21,mm_거래량_Day_22,mm_거래량_Day_23,mm_거래량_Day_24,mm_거래량_Day_25,mm_거래금액_만원단위_Day_1,mm_거래금액_만원단위_Day_2,mm_거래금액_만원단위_Day_3,mm_거래금액_만원단위_Day_4,mm_거래금액_만원단위_Day_5,mm_거래금액_만원단위_Day_6,mm_거래금액_만원단위_Day_7,mm_거래금액_만원단위_Day_8,mm_거래금액_만원단위_Day_9,mm_거래금액_만원단위_Day_10,mm_거래금액_만원단위_Day_11,mm_거래금액_만원단위_Day_12,mm_거래금액_만원단위_Day_13,mm_거래금액_만원단위_Day_14,mm_거래금액_만원단위_Day_15,mm_거래금액_만원단위_Day_16,mm_거래금액_만원단위_Day_17,mm_거래금액_만원단위_Day_18,mm_거래금액_만원단위_Day_19,mm_거래금액_만원단위_Day_20,mm_거래금액_만원단위_Day_21,mm_거래금액_만원단위_Day_22,mm_거래금액_만원단위_Day_23,mm_거래금액_만원단위_Day_24,mm_거래금액_만원단위_Day_25,std_종목시가_Day_1,std_종목시가_Day_2,std_종목시가_Day_3,std_종목시가_Day_4,std_종목시가_Day_5,std_종목시가_Day_6,std_종목시가_Day_7,std_종목시가_Day_8,std_종목시가_Day_9,std_종목시가_Day_10,std_종목시가_Day_11,std_종목시가_Day_12,std_종목시가_Day_13,std_종목시가_Day_14,std_종목시가_Day_15,std_종목시가_Day_16,std_종목시가_Day_17,std_종목시가_Day_18,std_종목시가_Day_19,std_종목시가_Day_20,std_종목시가_Day_21,std_종목시가_Day_22,std_종목시가_Day_23,std_종목시가_Day_24,std_종목시가_Day_25,std_종목고가_Day_1,std_종목고가_Day_2,std_종목고가_Day_3,std_종목고가_Day_4,std_종목고가_Day_5,std_종목고가_Day_6,std_종목고가_Day_7,std_종목고가_Day_8,std_종목고가_Day_9,std_종목고가_Day_10,std_종목고가_Day_11,std_종목고가_Day_12,std_종목고가_Day_13,std_종목고가_Day_14,std_종목고가_Day_15,std_종목고가_Day_16,std_종목고가_Day_17,std_종목고가_Day_18,std_종목고가_Day_19,std_종목고가_Day_20,std_종목고가_Day_21,std_종목고가_Day_22,std_종목고가_Day_23,std_종목고가_Day_24,std_종목고가_Day_25,std_종목저가_Day_1,std_종목저가_Day_2,std_종목저가_Day_3,std_종목저가_Day_4,std_종목저가_Day_5,std_종목저가_Day_6,std_종목저가_Day_7,std_종목저가_Day_8,std_종목저가_Day_9,std_종목저가_Day_10,std_종목저가_Day_11,std_종목저가_Day_12,std_종목저가_Day_13,std_종목저가_Day_14,std_종목저가_Day_15,std_종목저가_Day_16,std_종목저가_Day_17,std_종목저가_Day_18,std_종목저가_Day_19,std_종목저가_Day_20,std_종목저가_Day_21,std_종목저가_Day_22,std_종목저가_Day_23,std_종목저가_Day_24,std_종목저가_Day_25,std_종목종가_Day_1,std_종목종가_Day_2,std_종목종가_Day_3,std_종목종가_Day_4,std_종목종가_Day_5,std_종목종가_Day_6,std_종목종가_Day_7,std_종목종가_Day_8,std_종목종가_Day_9,std_종목종가_Day_10,std_종목종가_Day_11,std_종목종가_Day_12,std_종목종가_Day_13,std_종목종가_Day_14,std_종목종가_Day_15,std_종목종가_Day_16,std_종목종가_Day_17,std_종목종가_Day_18,std_종목종가_Day_19,std_종목종가_Day_20,std_종목종가_Day_21,std_종목종가_Day_22,std_종목종가_Day_23,std_종목종가_Day_24,std_종목종가_Day_25,std_거래량_Day_1,std_거래량_Day_2,std_거래량_Day_3,std_거래량

### 마트에 거래금액 등급 추가

In [66]:
def trade_10000(i, stocks=stocks, mart=mart):
    dt = stocks[(stocks['기준년월']==mart.iloc[i,0])&(stocks['종목번호']==mart.iloc[i,1])][['거래금액']].copy()
    dt['tf']=1
    add_dt = make_row25(dt).copy()
    add_dt['tf'] = (add_dt['거래금액']/10000000).astype(int)
    f = add_dt[['tf']].T.copy()
    f.columns = ['거래금액_class_Day'+str(i+1) for i in range(25)]
    return f

In [67]:
mart_n = len(mart)
class_10000=[0]*mart_n

for i in range(mart_n):
    if i%100==0:
        st_time = time.time()
        
    class_10000[i] = trade_10000(i)
    
    if i%100==0:
        ed_time = time.time()-st_time
        print("\r남은 개수 :",i,"/",mart_n,"\t\t남은 시간 :",round(ed_time*(mart_n-i-1),2),"\t\t",end='')

남은 개수 : 5900 / 5923 		남은 시간 : 1.4 					

In [68]:
class_10000 = pd.concat(class_10000)
class_10000.index = range(len(class_10000))
for i in ['거래금액_class_Day'+str(i+1) for i in range(25)]:
    mart[i] = class_10000[i].tolist()

In [69]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,철강,애국,한한령,대북,오일,무속성,mm_종목시가_Day_1,mm_종목시가_Day_2,mm_종목시가_Day_3,mm_종목시가_Day_4,mm_종목시가_Day_5,mm_종목시가_Day_6,mm_종목시가_Day_7,mm_종목시가_Day_8,mm_종목시가_Day_9,mm_종목시가_Day_10,mm_종목시가_Day_11,mm_종목시가_Day_12,mm_종목시가_Day_13,mm_종목시가_Day_14,mm_종목시가_Day_15,mm_종목시가_Day_16,mm_종목시가_Day_17,mm_종목시가_Day_18,mm_종목시가_Day_19,mm_종목시가_Day_20,mm_종목시가_Day_21,mm_종목시가_Day_22,mm_종목시가_Day_23,mm_종목시가_Day_24,mm_종목시가_Day_25,mm_종목고가_Day_1,mm_종목고가_Day_2,mm_종목고가_Day_3,mm_종목고가_Day_4,mm_종목고가_Day_5,mm_종목고가_Day_6,mm_종목고가_Day_7,mm_종목고가_Day_8,mm_종목고가_Day_9,mm_종목고가_Day_10,mm_종목고가_Day_11,mm_종목고가_Day_12,mm_종목고가_Day_13,mm_종목고가_Day_14,mm_종목고가_Day_15,mm_종목고가_Day_16,mm_종목고가_Day_17,mm_종목고가_Day_18,mm_종목고가_Day_19,mm_종목고가_Day_20,mm_종목고가_Day_21,mm_종목고가_Day_22,mm_종목고가_Day_23,mm_종목고가_Day_24,mm_종목고가_Day_25,mm_종목저가_Day_1,mm_종목저가_Day_2,mm_종목저가_Day_3,mm_종목저가_Day_4,mm_종목저가_Day_5,mm_종목저가_Day_6,mm_종목저가_Day_7,mm_종목저가_Day_8,mm_종목저가_Day_9,mm_종목저가_Day_10,mm_종목저가_Day_11,mm_종목저가_Day_12,mm_종목저가_Day_13,mm_종목저가_Day_14,mm_종목저가_Day_15,mm_종목저가_Day_16,mm_종목저가_Day_17,mm_종목저가_Day_18,mm_종목저가_Day_19,mm_종목저가_Day_20,mm_종목저가_Day_21,mm_종목저가_Day_22,mm_종목저가_Day_23,mm_종목저가_Day_24,mm_종목저가_Day_25,mm_종목종가_Day_1,mm_종목종가_Day_2,mm_종목종가_Day_3,mm_종목종가_Day_4,mm_종목종가_Day_5,mm_종목종가_Day_6,mm_종목종가_Day_7,mm_종목종가_Day_8,mm_종목종가_Day_9,mm_종목종가_Day_10,mm_종목종가_Day_11,mm_종목종가_Day_12,mm_종목종가_Day_13,mm_종목종가_Day_14,mm_종목종가_Day_15,mm_종목종가_Day_16,mm_종목종가_Day_17,mm_종목종가_Day_18,mm_종목종가_Day_19,mm_종목종가_Day_20,mm_종목종가_Day_21,mm_종목종가_Day_22,mm_종목종가_Day_23,mm_종목종가_Day_24,mm_종목종가_Day_25,mm_거래량_Day_1,mm_거래량_Day_2,mm_거래량_Day_3,mm_거래량_Day_4,mm_거래량_Day_5,mm_거래량_Day_6,mm_거래량_Day_7,mm_거래량_Day_8,mm_거래량_Day_9,mm_거래량_Day_10,mm_거래량_Day_11,mm_거래량_Day_12,mm_거래량_Day_13,mm_거래량_Day_14,mm_거래량_Day_15,mm_거래량_Day_16,mm_거래량_Day_17,mm_거래량_Day_18,mm_거래량_Day_19,mm_거래량_Day_20,mm_거래량_Day_21,mm_거래량_Day_22,mm_거래량_Day_23,mm_거래량_Day_24,mm_거래량_Day_25,mm_거래금액_만원단위_Day_1,mm_거래금액_만원단위_Day_2,mm_거래금액_만원단위_Day_3,mm_거래금액_만원단위_Day_4,mm_거래금액_만원단위_Day_5,mm_거래금액_만원단위_Day_6,mm_거래금액_만원단위_Day_7,mm_거래금액_만원단위_Day_8,mm_거래금액_만원단위_Day_9,mm_거래금액_만원단위_Day_10,mm_거래금액_만원단위_Day_11,mm_거래금액_만원단위_Day_12,mm_거래금액_만원단위_Day_13,mm_거래금액_만원단위_Day_14,mm_거래금액_만원단위_Day_15,mm_거래금액_만원단위_Day_16,mm_거래금액_만원단위_Day_17,mm_거래금액_만원단위_Day_18,mm_거래금액_만원단위_Day_19,mm_거래금액_만원단위_Day_20,mm_거래금액_만원단위_Day_21,mm_거래금액_만원단위_Day_22,mm_거래금액_만원단위_Day_23,mm_거래금액_만원단위_Day_24,mm_거래금액_만원단위_Day_25,std_종목시가_Day_1,std_종목시가_Day_2,std_종목시가_Day_3,std_종목시가_Day_4,std_종목시가_Day_5,std_종목시가_Day_6,std_종목시가_Day_7,std_종목시가_Day_8,std_종목시가_Day_9,std_종목시가_Day_10,std_종목시가_Day_11,std_종목시가_Day_12,std_종목시가_Day_13,std_종목시가_Day_14,std_종목시가_Day_15,std_종목시가_Day_16,std_종목시가_Day_17,std_종목시가_Day_18,std_종목시가_Day_19,std_종목시가_Day_20,std_종목시가_Day_21,std_종목시가_Day_22,std_종목시가_Day_23,std_종목시가_Day_24,std_종목시가_Day_25,std_종목고가_Day_1,std_종목고가_Day_2,std_종목고가_Day_3,std_종목고가_Day_4,std_종목고가_Day_5,std_종목고가_Day_6,std_종목고가_Day_7,std_종목고가_Day_8,std_종목고가_Day_9,std_종목고가_Day_10,std_종목고가_Day_11,std_종목고가_Day_12,std_종목고가_Day_13,std_종목고가_Day_14,std_종목고가_Day_15,std_종목고가_Day_16,std_종목고가_Day_17,std_종목고가_Day_18,std_종목고가_Day_19,std_종목고가_Day_20,std_종목고가_Day_21,std_종목고가_Day_22,std_종목고가_Day_23,std_종목고가_Day_24,std_종목고가_Day_25,std_종목저가_Day_1,std_종목저가_Day_2,std_종목저가_Day_3,std_종목저가_Day_4,std_종목저가_Day_5,std_종목저가_Day_6,std_종목저가_Day_7,std_종목저가_Day_8,std_종목저가_Day_9,std_종목저가_Day_10,std_종목저가_Day_11,std_종목저가_Day_12,std_종목저가_Day_13,std_종목저가_Day_14,std_종목저가_Day_15,std_종목저가_Day_16,std_종목저가_Day_17,std_종목저가_Day_18,std_종목저가_Day_19,std_종목저가_Day_20,std_종목저가_Day_21,std_종목저가_Day_22,std_종목저가_Day_23,std_종목저가_Day_24,std_종목저가_Day_25,std_종목종가_Day_1,std_종목종가_Day_2,std_종목종가_Day_3,std_종목종가_Day_4,std_종목종가_Day_5,std_종목종가_Day_6,std_종목종가_Day_7,std_종목종가_Day_8,std_종목종가_Day_9,std_종목종가_Day_10,std_종목종가_Day_11,std_종목종가_Day_12,std_종목종가_Day_13,std_종목종가_Day_14,std_종목종가_Day_15,std_종목종가_Day_16,std_종목종가_Day_17,std_종목종가_Day_18,std_종목종가_Day_19,std_종목종가_Day_20,std_종목종가_Day_21,std_종목종가_Day_22,std_종목종가_Day_23,std_종목종가_Day_24,std_종목종가_Day_25,std_거래량_Day_1,std_거래량_Day_2,std_거래량_Day_3,std_거래량

### 마트에 거래금액 값 추가

In [70]:
def trade_10000_2(i, stocks=stocks, mart=mart):
    dt = stocks[(stocks['기준년월']==mart.iloc[i,0])&(stocks['종목번호']==mart.iloc[i,1])][['거래금액']].copy()
    dt['tf']=1
    add_dt = make_row25(dt).copy()
    add_dt['tf'] = (add_dt['거래금액']/1000000000)
    f = add_dt[['tf']].T.copy()
    f.columns = ['거래금액_float_Day'+str(i+1) for i in range(25)]
    return f

In [71]:
mart_n = len(mart)
float_10000=[0]*mart_n

for i in range(mart_n):
    if i%100==0:
        st_time = time.time()
        
    float_10000[i] = trade_10000_2(i)
    
    if i%100==0:
        ed_time = time.time()-st_time
        print("\r",i,"/",mart_n,"\t",round(ed_time*(mart_n-i-1),2),"\t\t",end='')

 5900 / 5923 	 2.28 				

In [72]:
float_10000 = pd.concat(float_10000)
float_10000.index = range(len(float_10000))
for i in ['거래금액_float_Day'+str(i+1) for i in range(25)]:
    mart[i] = float_10000[i].tolist()

In [73]:
mart.head()

,기준년월,종목번호,종목명,시장구분,대분류,중분류,소분류,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,철강,애국,한한령,대북,오일,무속성,mm_종목시가_Day_1,mm_종목시가_Day_2,mm_종목시가_Day_3,mm_종목시가_Day_4,mm_종목시가_Day_5,mm_종목시가_Day_6,mm_종목시가_Day_7,mm_종목시가_Day_8,mm_종목시가_Day_9,mm_종목시가_Day_10,mm_종목시가_Day_11,mm_종목시가_Day_12,mm_종목시가_Day_13,mm_종목시가_Day_14,mm_종목시가_Day_15,mm_종목시가_Day_16,mm_종목시가_Day_17,mm_종목시가_Day_18,mm_종목시가_Day_19,mm_종목시가_Day_20,mm_종목시가_Day_21,mm_종목시가_Day_22,mm_종목시가_Day_23,mm_종목시가_Day_24,mm_종목시가_Day_25,mm_종목고가_Day_1,mm_종목고가_Day_2,mm_종목고가_Day_3,mm_종목고가_Day_4,mm_종목고가_Day_5,mm_종목고가_Day_6,mm_종목고가_Day_7,mm_종목고가_Day_8,mm_종목고가_Day_9,mm_종목고가_Day_10,mm_종목고가_Day_11,mm_종목고가_Day_12,mm_종목고가_Day_13,mm_종목고가_Day_14,mm_종목고가_Day_15,mm_종목고가_Day_16,mm_종목고가_Day_17,mm_종목고가_Day_18,mm_종목고가_Day_19,mm_종목고가_Day_20,mm_종목고가_Day_21,mm_종목고가_Day_22,mm_종목고가_Day_23,mm_종목고가_Day_24,mm_종목고가_Day_25,mm_종목저가_Day_1,mm_종목저가_Day_2,mm_종목저가_Day_3,mm_종목저가_Day_4,mm_종목저가_Day_5,mm_종목저가_Day_6,mm_종목저가_Day_7,mm_종목저가_Day_8,mm_종목저가_Day_9,mm_종목저가_Day_10,mm_종목저가_Day_11,mm_종목저가_Day_12,mm_종목저가_Day_13,mm_종목저가_Day_14,mm_종목저가_Day_15,mm_종목저가_Day_16,mm_종목저가_Day_17,mm_종목저가_Day_18,mm_종목저가_Day_19,mm_종목저가_Day_20,mm_종목저가_Day_21,mm_종목저가_Day_22,mm_종목저가_Day_23,mm_종목저가_Day_24,mm_종목저가_Day_25,mm_종목종가_Day_1,mm_종목종가_Day_2,mm_종목종가_Day_3,mm_종목종가_Day_4,mm_종목종가_Day_5,mm_종목종가_Day_6,mm_종목종가_Day_7,mm_종목종가_Day_8,mm_종목종가_Day_9,mm_종목종가_Day_10,mm_종목종가_Day_11,mm_종목종가_Day_12,mm_종목종가_Day_13,mm_종목종가_Day_14,mm_종목종가_Day_15,mm_종목종가_Day_16,mm_종목종가_Day_17,mm_종목종가_Day_18,mm_종목종가_Day_19,mm_종목종가_Day_20,mm_종목종가_Day_21,mm_종목종가_Day_22,mm_종목종가_Day_23,mm_종목종가_Day_24,mm_종목종가_Day_25,mm_거래량_Day_1,mm_거래량_Day_2,mm_거래량_Day_3,mm_거래량_Day_4,mm_거래량_Day_5,mm_거래량_Day_6,mm_거래량_Day_7,mm_거래량_Day_8,mm_거래량_Day_9,mm_거래량_Day_10,mm_거래량_Day_11,mm_거래량_Day_12,mm_거래량_Day_13,mm_거래량_Day_14,mm_거래량_Day_15,mm_거래량_Day_16,mm_거래량_Day_17,mm_거래량_Day_18,mm_거래량_Day_19,mm_거래량_Day_20,mm_거래량_Day_21,mm_거래량_Day_22,mm_거래량_Day_23,mm_거래량_Day_24,mm_거래량_Day_25,mm_거래금액_만원단위_Day_1,mm_거래금액_만원단위_Day_2,mm_거래금액_만원단위_Day_3,mm_거래금액_만원단위_Day_4,mm_거래금액_만원단위_Day_5,mm_거래금액_만원단위_Day_6,mm_거래금액_만원단위_Day_7,mm_거래금액_만원단위_Day_8,mm_거래금액_만원단위_Day_9,mm_거래금액_만원단위_Day_10,mm_거래금액_만원단위_Day_11,mm_거래금액_만원단위_Day_12,mm_거래금액_만원단위_Day_13,mm_거래금액_만원단위_Day_14,mm_거래금액_만원단위_Day_15,mm_거래금액_만원단위_Day_16,mm_거래금액_만원단위_Day_17,mm_거래금액_만원단위_Day_18,mm_거래금액_만원단위_Day_19,mm_거래금액_만원단위_Day_20,mm_거래금액_만원단위_Day_21,mm_거래금액_만원단위_Day_22,mm_거래금액_만원단위_Day_23,mm_거래금액_만원단위_Day_24,mm_거래금액_만원단위_Day_25,std_종목시가_Day_1,std_종목시가_Day_2,std_종목시가_Day_3,std_종목시가_Day_4,std_종목시가_Day_5,std_종목시가_Day_6,std_종목시가_Day_7,std_종목시가_Day_8,std_종목시가_Day_9,std_종목시가_Day_10,std_종목시가_Day_11,std_종목시가_Day_12,std_종목시가_Day_13,std_종목시가_Day_14,std_종목시가_Day_15,std_종목시가_Day_16,std_종목시가_Day_17,std_종목시가_Day_18,std_종목시가_Day_19,std_종목시가_Day_20,std_종목시가_Day_21,std_종목시가_Day_22,std_종목시가_Day_23,std_종목시가_Day_24,std_종목시가_Day_25,std_종목고가_Day_1,std_종목고가_Day_2,std_종목고가_Day_3,std_종목고가_Day_4,std_종목고가_Day_5,std_종목고가_Day_6,std_종목고가_Day_7,std_종목고가_Day_8,std_종목고가_Day_9,std_종목고가_Day_10,std_종목고가_Day_11,std_종목고가_Day_12,std_종목고가_Day_13,std_종목고가_Day_14,std_종목고가_Day_15,std_종목고가_Day_16,std_종목고가_Day_17,std_종목고가_Day_18,std_종목고가_Day_19,std_종목고가_Day_20,std_종목고가_Day_21,std_종목고가_Day_22,std_종목고가_Day_23,std_종목고가_Day_24,std_종목고가_Day_25,std_종목저가_Day_1,std_종목저가_Day_2,std_종목저가_Day_3,std_종목저가_Day_4,std_종목저가_Day_5,std_종목저가_Day_6,std_종목저가_Day_7,std_종목저가_Day_8,std_종목저가_Day_9,std_종목저가_Day_10,std_종목저가_Day_11,std_종목저가_Day_12,std_종목저가_Day_13,std_종목저가_Day_14,std_종목저가_Day_15,std_종목저가_Day_16,std_종목저가_Day_17,std_종목저가_Day_18,std_종목저가_Day_19,std_종목저가_Day_20,std_종목저가_Day_21,std_종목저가_Day_22,std_종목저가_Day_23,std_종목저가_Day_24,std_종목저가_Day_25,std_종목종가_Day_1,std_종목종가_Day_2,std_종목종가_Day_3,std_종목종가_Day_4,std_종목종가_Day_5,std_종목종가_Day_6,std_종목종가_Day_7,std_종목종가_Day_8,std_종목종가_Day_9,std_종목종가_Day_10,std_종목종가_Day_11,std_종목종가_Day_12,std_종목종가_Day_13,std_종목종가_Day_14,std_종목종가_Day_15,std_종목종가_Day_16,std_종목종가_Day_17,std_종목종가_Day_18,std_종목종가_Day_19,std_종목종가_Day_20,std_종목종가_Day_21,std_종목종가_Day_22,std_종목종가_Day_23,std_종목종가_Day_24,std_종목종가_Day_25,std_거래량_Day_1,std_거래량_Day_2,std_거래량_Day_3,std_거래량

# 코로나 확산 이후 매수고객 수 Top3에 뽑힌 횟수
* 코로나로 인한 주식 시장 흐름의 변화가 크다고 판단
* 매수고객 수가 많았었던 종목이 그 그룹의 경향을 나타낸다고 생각

In [74]:
corona_month = [202003+i for i in range(4)]

In [75]:
mart_ans = trade[['그룹번호']].drop_duplicates().sort_values('그룹번호').copy()
mart_ans.index = range(len(mart_ans))
group_list = mart_ans['그룹번호'].tolist()

In [76]:
for i in group_list:
    mart[i] = 0

In [77]:
for i in range(48):
    ovr = [trade[(trade['기준년월']==ik)&(trade['그룹번호']==group_list[i])].sort_values('매수고객수',ascending=False).copy() for ik in corona_month]
    ovr = [ik[ik['종목번호'].apply(lambda x: True if x in y_list else False)].head(3)['종목번호'].tolist() for ik in ovr]
    
    for cnt in ovr:
        mart.loc[mart['종목번호'].apply(lambda x: True if x in cnt else False),group_list[i]] += 1

# 정답 데이터프레임 생성

### mart의 row 별로 그룹마다 매수고객수(목표값) 데이터프레임 생성

In [78]:
def ans_value(i,trade=trade,m=mart,mart_ans=mart_ans):
    mart = m[m['기준년월']<202007].copy()
    ans_val = pd.merge(mart_ans,trade[(trade['기준년월']==mart.iloc[i,0])&(trade['종목번호']==mart.iloc[i,1])][['그룹번호','매수고객수']],on=['그룹번호'],how='left')[['매수고객수']]
    ans_val = ans_val.fillna(0)
    ans_val['매수고객수'] = ans_val['매수고객수'].astype(int)
    return ans_val

In [79]:
mart_n = len(mart[mart['기준년월']<202007])
ans_val = [0]*mart_n
for i in range(mart_n):
    if i%100==0:
        st_time = time.time()
        
    ans_val[i] = ans_value(i).T
    
    if i%100==0:
        ed_time = time.time()-st_time
        print("\r남은 개수 :",i,"/",mart_n,"\t\t남은 시간 :",round(ed_time*(mart_n-i-1),2),"\t\t",end='')

남은 개수 : 5700 / 5791 		남은 시간 : 5.39 				

In [80]:
ans_val = pd.concat(ans_val)
ans_val.index = range(len(ans_val))

In [81]:
ans_val.columns = group_list

In [82]:
ans_val.head()

,MAD01,MAD02,MAD03,MAD04,MAD05,MAD06,MAD07,MAD08,MAD09,MAD10,MAD11,MAD12,MAD13,MAD14,MAD15,MAD16,MAD17,MAD18,MAD19,MAD20,MAD21,MAD22,MAD23,MAD24,MAD25,MAD26,MAD27,MAD28,MAD29,MAD30,MAD31,MAD32,MAD33,MAD34,MAD35,MAD36,MAD37,MAD38,MAD39,MAD40,MAD41,MAD42,MAD43,MAD44,MAD45,MAD46,MAD47,MAD48
0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0
1,0,0,0,8,4,9,0,0,0,0,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,0,0,0,0,3,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0


# Pycaret을 활용한 머신러닝 모델 선택

In [83]:
train_X = mart[mart['기준년월']<202007].iloc[:,3:].copy()
train_Y = ans_val.copy()
test_X = mart[mart['기준년월']==202007].iloc[:,3:].copy()
test_X.index = range(len(test_X))
print(len(train_X)==len(train_Y))

True


In [84]:
x = train_X.copy()
x['target'] = train_Y.iloc[:,0].copy()

In [86]:
exp = setup(x, target='target',verbose=False, session_id=1192)
compare_models(exclude=['ransac','tr','catboost'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.4124,3.9598,1.8840,0.5765,0.3390,0.7617,21.2840
gbr,Gradient Boosting Regressor,0.4608,4.8066,2.0557,0.4764,0.3571,0.7512,25.6460
rf,Random Forest Regressor,0.4550,4.1526,1.9608,0.4752,0.3807,0.7625,83.2380
xgboost,Extreme Gradient Boosting,0.4543,4.4164,2.0372,0.3798,0.3544,0.7655,26.4350
lightgbm,Light Gradient Boosting Machine,0.5318,5.0326,2.1572,0.3569,0.3945,0.7913,3.3100
knn,K Neighbors Regressor,0.4466,8.7454,2.7009,0.2366,0.4182,0.8903,1.9660
dt,Decision Tree Regressor,0.4534,6.5896,2.4762,0.1308,0.4290,1.0107,1.7530
huber,Huber Regressor,0.3976,11.2995,3.1007,0.0015,0.4102,0.9876,3.8320
en,Elastic Net,0.7592,11.2793,3.1005,0.0001,0.4924,0.9200,0.1290
lasso,Lasso Regression,0.7601,11.2915,3.1025,-0.0016,0.4928,0.9217,0.1220


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=1192, verbose=0, warm_start=False)

# Extra Trees Regressor 머신러닝 모델 사용

In [87]:
model = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=1192, verbose=0, warm_start=False)

In [88]:
model.fit(train_X, train_Y)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=1192, verbose=0, warm_start=False)

In [89]:
result = model.predict(test_X)
result = pd.DataFrame(result).copy()

In [90]:
result.columns = group_list
for i in ['종목번호','종목명']+thema:
    result[i] = mart[mart['기준년월']==202007][i].tolist()
result = result[['종목번호','종목명']+thema+group_list].copy()

In [91]:
result.head(3)

,종목번호,종목명,대선,코로나,언택트,백신,미래산업,인플루언서,미중무역,돼지열병_흑사병,...,MAD39,MAD40,MAD41,MAD42,MAD43,MAD44,MAD45,MAD46,MAD47,MAD48
0,A000100,유한양행,0,0,0,1,0,0,0,0,...,0.18,0.13,15.83,6.05,8.00,0.10,0.04,2.02,0.06,0.49
1,A000140,하이트진로홀딩스,0,0,0,0,0,0,0,0,...,0.00,0.00,0.73,0.00,0.51,0.03,0.00,0.11,0.00,0.09
2,A000440,중앙에너비스,1,0,0,0,0,0,0,0,...,0.00,0.00,1.23,0.16,0.76,0.00,0.00,0.03,0.06,0.46


In [92]:
result_temp = result.copy()

In [93]:
result = result_temp.copy()

# 그룹별 특성을 파악한 가중치 List

각 그룹의 경향성을 파악하여 가중치를 부여할 List를 생성

In [94]:
group_thema = []
group_thema.append([]) # 1
group_thema.append(['무속성']) # 2
group_thema.append(['백신']) # 3
group_thema.append(['백신']) # 4
group_thema.append(['언택트']) # 5
group_thema.append(['언택트']) # 6
group_thema.append(['오일']) # 7
group_thema.append(['백신']) # 8
group_thema.append(['언택트']) # 9
group_thema.append(['미래산업']) # 10
group_thema.append(['인플루언서']) # 11
group_thema.append(['언택트']) # 12
group_thema.append([]) # 13
group_thema.append(['인플루언서']) # 14
group_thema.append(['미래산업']) # 15
group_thema.append([]) # 16
group_thema.append(['미래산업']) # 17
group_thema.append([]) # 18
group_thema.append(['미래산업']) # 19
group_thema.append(['언택트']) # 20
group_thema.append(['언택트']) # 21
group_thema.append(['백신']) # 22
group_thema.append(['미래산업']) # 23
group_thema.append(['인플루언서']) # 24
group_thema.append([]) # 25
group_thema.append(['인플루언서']) # 26
group_thema.append(['미래산업']) # 27
group_thema.append(['인플루언서']) # 28
group_thema.append(['미래산업']) # 29
group_thema.append([]) # 30
group_thema.append(['미래산업']) # 31
group_thema.append(['미래산업']) # 32
group_thema.append(['미래산업']) # 33
group_thema.append(['미래산업']) # 34
group_thema.append(['인플루언서']) # 35
group_thema.append(['코로나']) # 36
group_thema.append(['언택트']) # 37
group_thema.append(['언택트']) # 38
group_thema.append(['백신']) # 39
group_thema.append(['백신']) # 40
group_thema.append(['백신']) # 41
group_thema.append(['백신']) # 42
group_thema.append(['무속성']) # 43
group_thema.append([]) # 44
group_thema.append([]) # 45
group_thema.append(['인플루언서']) # 46
group_thema.append(['미래산업']) # 47
group_thema.append(['미래산업']) # 48

# 최종 Top3 산출

In [95]:
th_w = 7
ans_list = [0]*48
for i in range(48):
    ans = sorted(mart[mart[group_list[i]]==4]['종목번호'].unique().tolist())
    if len(ans)!=3:
        r = result.copy()
        cnt = 0
        for gr_th in group_thema[i]:
            if gr_th in ['미래산업','인플루언서']:
                r.loc[(r[gr_th]==1),group_list[i]] *= th_w
            else:
                r.loc[(r[gr_th]==1)&(r[thema].sum(axis=1)==1),group_list[i]] *= th_w
        while len(ans)!=3:
            ans.append(
                r.loc[:,['종목번호',group_list[i]]].sort_values(group_list[i],ascending=False).iloc[cnt,0]
            )
            cnt += 1
            ans = sorted(list(set(ans)))
    ans_list[i] = ans

# Answer_sheet 제작 과정

In [96]:
ans_list = pd.DataFrame(ans_list)
final_ans = mart_ans.copy()
for i in range(3):
    final_ans[i] = ans_list[i].tolist()
final_ans.columns = ['그룹번호']+['종목번호'+str(i+1) for i in range(3)]

In [97]:
final_ans.to_csv("Topology_answer.csv",index=False)